In [1]:
import smf2021.rank2 as rank2
import smf2021.Optimiser as O
from smf.db_engine import DbEngine
import eikon as ek
# from smf.ranker import retrieve
# from smf.ranker import rank
# from smf.ranker import add_sector_info
from smf.db_engine import DbEngine
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

from numpy import log
from dateutil.parser import parse
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import pmdarima as pm
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import time
import datetime as dt


In [2]:
# Industrials
ind_fields = ['TR.GICSSector','TR.PE',"TR.PriceToBVPerShare",
              "TR.EPSMean", "TR.EVToSales", "TR.EVToEBIT",
              "TR.PtoEBTMeanEst", "TR.FwdPEG"]


# Comms
comms_fields = ['TR.GICSSector','TR.ROATotalAssetsPercent',
                'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                'TR.EVToEBITDA',
                'TR.ROICActValue', 'TR.TimesInterestEarned', 'TR.FCFMeanYield',
               ]

# Healthcare
health_fields = ['TR.GICSSector','TR.PE',  'TR.EVToEBITDA',
                 'TR.EVToSales', 'TR.ROICActValue', 'TR.FCFMeanYield',
'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
                 "TR.PriceToBVPerShare"]

cons_fields = ['TR.GICSSector','TR.PE', 'TR.ROATotalAssetsPercent', 
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.EVToEBITDA', 'TR.EVToSales']

# fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", "TR.EPSMean",
#           "TR.EVToSales", "TR.EVToEBIT",
#           "TR.PtoEBTMeanEst", "TR.FwdPEG"]  
# Tech
tech_fields = ['TR.GICSSector','TR.PE', "TR.PriceToBVPerShare", 
               'TR.ROICActValue', 'TR.ReinvestmentRatePct',
               'TR.ReturnonAVGCommEqtyPctIncomeAvailabletoCommExclExtraItems',
               'TR.FCFMeanYield']

In [3]:
ind_ranker = rank2.ranker(ind_fields)
comms_ranker = rank2.ranker(comms_fields)
health_ranker = rank2.ranker(health_fields)
cons_ranker = rank2.ranker(cons_fields)
tech_ranker = rank2.ranker(tech_fields)
all_comms = comms_ranker.get_data()
all_ind = ind_ranker.get_data()
all_health = health_ranker.get_data()
all_cons = cons_ranker.get_data()
all_tech = tech_ranker.get_data()
all_tech = tech_ranker.get_data()

In [4]:
# True => lower is better.
# ROIC < WACC => remove from list.
# Interest Coverage Ratio > 1.5
ind_data = ind_ranker.industry_data('Industrials')
comms_data = comms_ranker.industry_data('Communication Services')
cons_data = cons_ranker.industry_data('Consumer Staples')
health_data = health_ranker.industry_data('Health Care')
tech_data = health_ranker.industry_data('Information Technology')
asc_ind = {'P/E (Daily Time Series Ratio)': True,
           'Price To Book Value Per Share (Daily Time Series Ratio)': True,
           'Earnings Per Share - Mean': False, 
           'Enterprise Value To Sales (Daily Time Series Ratio)': True,
           'Enterprise Value To EBIT (Daily Time Series Ratio)': False, 
           'Price / EBITDA (Mean Estimate)': True, 
           'Forward P/E/G (Daily Time Series Ratio)': True}
             
asc_comms = {'ROA Total Assets, Percent': False,
             'ROE Common Equity %': False,
             'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
             'Return On Invested Capital - Actual': False,
             'Interest Coverage Ratio': False,
             'Free Cash Flow Yield %': False}


asc_tech = {'P/E (Daily Time Series Ratio)': True,
            'ROE Common Equity %': False,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True,

            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False}

asc_health = {'P/E (Daily Time Series Ratio)': True,
            'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
            'Enterprise Value To Sales (Daily Time Series Ratio)': True,
            'Return On Invested Capital - Actual': False,
            'Free Cash Flow Yield %': False,
            'ROE Common Equity %': False,
            'Price To Book Value Per Share (Daily Time Series Ratio)': True}

asc_cons = {'P/E (Daily Time Series Ratio)': True,
                'ROA Total Assets, Percent': False,
                'ROE Common Equity %': False,
                'Enterprise Value To EBITDA (Daily Time Series Ratio)': True,
                'Enterprise Value To Sales (Daily Time Series Ratio)': True
                }

In [16]:
ind_z_scores = ind_ranker.z_scores(ind_data, asc_ind)
health_z_scores = health_ranker.z_scores(health_data, asc_health)
comms_z_scores = comms_ranker.z_scores(comms_data, asc_comms)
tech_z_scores = tech_ranker.z_scores(tech_data, asc_tech)
cons_z_scores = cons_ranker.z_scores(cons_data, asc_cons)
cons_list = cons_z_scores.head(5)['Instrument'].tolist()
ind_list = ind_z_scores.head(5)['Instrument'].tolist()
health_list = health_z_scores.head(5)['Instrument'].tolist()
comms_list = comms_z_scores.head(5)["Instrument"].tolist()
tech_list = tech_z_scores.head(5)["Instrument"].tolist()

stocks_list = cons_list+comms_list+ind_list+health_list+tech_list
len(stocks_list)


25

Optimiser 1

In [17]:
final_companies1 = stocks_list
data=ek.get_timeseries(final_companies1, fields='CLOSE', start_date='2021-01-19', end_date='2022-01-19' )
close_prices_12m = data.fillna(method="ffill")
close_prices_12m =close_prices_12m ._get_numeric_data()
close_prices_12m=close_prices_12m.astype(float)

In [18]:
close_prices_12m

CLOSE,KMB.N,CL.N,TSN.N,PM.N,CPB.N,GOOGL.OQ,GOOG.OQ,LUMN.N,DISH.OQ,FB.OQ,...,OGN.N,HCA.N,MCK.N,CAH.N,REGN.OQ,MSI.N,HPQ.N,HPE.N,ADSK.OQ,STX.OQ
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-30,135.72,79.50,71.46,100.09,43.72,2694.53,2704.42,12.47,41.89,356.30,...,29.01,248.20,203.83,59.38,574.61,223.92,28.87,14.50,321.13,87.90
2021-08-02,134.67,79.54,71.75,100.03,43.31,2697.09,2719.79,12.54,41.69,351.95,...,29.53,251.00,203.57,59.57,577.13,223.93,29.14,14.49,322.07,88.50
2021-08-03,135.64,79.42,71.71,100.44,43.35,2712.60,2725.60,12.75,42.00,351.24,...,29.51,253.51,206.42,59.63,586.51,224.99,29.53,14.85,327.61,90.42
2021-08-04,133.70,78.25,70.36,99.49,42.45,2702.51,2720.57,11.62,41.51,358.92,...,28.63,248.26,205.37,58.98,580.95,222.77,29.07,14.66,329.09,89.32
2021-08-05,133.41,78.90,70.14,99.59,42.23,2725.03,2738.80,11.78,41.84,362.97,...,29.56,249.06,199.67,50.57,602.47,222.27,29.23,14.69,328.82,90.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-12,143.09,82.87,90.75,101.44,45.21,2828.61,2832.96,13.29,36.37,333.26,...,32.61,249.30,254.63,51.68,631.44,256.85,38.99,17.15,269.60,113.23
2022-01-13,142.89,82.54,93.21,102.10,45.88,2771.74,2782.62,12.27,36.30,326.48,...,32.90,254.41,254.75,52.24,616.11,250.90,38.70,17.49,260.17,110.84
2022-01-14,143.20,83.45,93.76,103.38,46.38,2789.61,2795.73,12.34,36.19,331.90,...,32.86,257.00,256.52,53.09,615.24,251.13,38.42,17.56,259.10,109.20


In [19]:
min_weight, max_weight = 0.01, 0.20
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.01, 0.2]

Expected annual return: 42.8%
Annual volatility: 12.7%
Sharpe Ratio: 3.22
TSN.N       0.20000
HPQ.N       0.20000
MCK.N       0.20000
NOC.N       0.08906
REGN.OQ     0.06181
CL.N        0.04018
KMB.N       0.02142
MAS.N       0.01753
OGN.N       0.01000
ADSK.OQ     0.01000
HPE.N       0.01000
MSI.N       0.01000
CAH.N       0.01000
HCA.N       0.01000
FDX.N       0.01000
HII.N       0.01000
URI.N       0.01000
FB.OQ       0.01000
DISH.OQ     0.01000
LUMN.N      0.01000
GOOG.OQ     0.01000
GOOGL.OQ    0.01000
CPB.N       0.01000
PM.N        0.01000
STX.OQ      0.01000
dtype: float64


In [20]:
print(pd.Series(cleaned_weights).sort_values(ascending=False)[:8])

TSN.N      0.20000
HPQ.N      0.20000
MCK.N      0.20000
NOC.N      0.08906
REGN.OQ    0.06181
CL.N       0.04018
KMB.N      0.02142
MAS.N      0.01753
dtype: float64


In [21]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [22]:
curr_prices

Instrument
KMB.N        130.95
CL.N          79.72
TSN.N         97.88
PM.N         105.07
CPB.N         43.19
GOOGL.OQ     2772.4
GOOG.OQ     2772.05
LUMN.N        10.83
DISH.OQ       31.93
FB.OQ        228.07
MAS.N         58.27
URI.N        325.68
FDX.N        240.41
NOC.N        382.02
HII.N        179.65
OGN.N         34.45
HCA.N        240.01
MCK.N        273.53
CAH.N         52.81
REGN.OQ      618.22
MSI.N        221.86
HPQ.N         38.11
HPE.N         17.35
ADSK.OQ      238.52
STX.OQ       110.08
Name: CF_LAST, dtype: Float64

In [23]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

Left Over: 128.88999999999942


In [24]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
KMB.N,10,0.02142,130.95
CL.N,31,0.04018,79.72
TSN.N,123,0.20000,97.88
PM.N,6,0.01000,105.07
CPB.N,14,0.01000,43.19
LUMN.N,56,0.01000,10.83
DISH.OQ,19,0.01000,31.93
FB.OQ,3,0.01000,228.07
MAS.N,19,0.01753,58.27
URI.N,2,0.01000,325.68


In [25]:
(df['Number of shares'] * df['Current Price']).sum()
# df.to_csv("optimisation1.csv")

59871.11000000001